In [12]:
Anime.objects.count()

266

In [14]:
Manga.objects.count()

3857

In [17]:
NB_MANGA_LIMIT = 20
NB_ANIME_LIMIT = 20
NB_USERS_LIMIT = 200

SearchIssue.objects.all().delete()
# Profile.objects.all().delete()  # Keeping some profiles would be better
Suggestion.objects.all().delete()
Recommendation.objects.all().delete()
Pairing.objects.all().delete()
SocialApp.objects.all().delete()
Session.objects.all().delete()
ContentType.objects.all().delete()

(0, {})

In [18]:
from collections import Counter
nb = Counter(Rating.objects.values_list('work_id', flat=True))
# Distinction between Anime and Manga will disappear with PR #153
if NB_ANIME_LIMIT:
    work_ids = list(Anime.objects.values_list('id', flat=True))
    work_ids.sort(key=lambda work_id: -nb[work_id])
    Anime.objects.exclude(id__in=work_ids[:NB_ANIME_LIMIT]).delete()
if NB_MANGA_LIMIT:
    work_ids = list(Manga.objects.values_list('id', flat=True))
    work_ids.sort(key=lambda work_id: -nb[work_id])
    Manga.objects.exclude(id__in=work_ids[:NB_MANGA_LIMIT]).delete()

In [20]:
bundle = []
for artist_ids in Work.objects.values_list('artists'):
    bundle.extend(artist_ids)
kept_artist_ids = list(set([x for x in bundle if x is not None]))
Artist.objects.exclude(id__in=kept_artist_ids).delete()

(2383,
 {'mangaki.Album': 1,
  'mangaki.Artist': 2339,
  'mangaki.Pairing': 0,
  'mangaki.Rating': 42,
  'mangaki.Recommendation': 0,
  'mangaki.Staff': 0,
  'mangaki.Track_album': 0,
  'mangaki.Work': 1})

In [21]:
Artist.objects.count()

54

In [22]:
Work.objects.count()

40

In [23]:
Rating.objects.count()

28989

In [24]:
max_user_id = max(User.objects.values_list('id', flat=True))

In [25]:
import random
chosen = User.objects.order_by('?')[:NB_USERS_LIMIT]
kept_ids = chosen.values_list('id', flat=True)
User.objects.exclude(id__in=kept_ids).delete()

(50543,
 {'account.EmailAddress': 1672,
  'account.EmailConfirmation': 1654,
  'admin.LogEntry': 581,
  'auth.User': 1687,
  'auth.User_groups': 3,
  'auth.User_user_permissions': 0,
  'irl.Attendee': 10,
  'mangaki.Neighborship': 17330,
  'mangaki.Pairing': 0,
  'mangaki.Profile': 1684,
  'mangaki.Rating': 25891,
  'mangaki.Recommendation': 0,
  'mangaki.SearchIssue': 0,
  'socialaccount.SocialAccount': 31,
  'socialaccount.SocialToken': 0})

In [27]:
User.objects.count()

200

In [28]:
from django.utils import timezone
new_ids = random.sample(range(max_user_id + 1, max_user_id + 201), 200)
for user, new_id in zip(User.objects.all(), new_ids):
    old_id = user.id
    user.pk = new_id
    user.username = str(new_id)
    user.is_superuser = False
    user.set_password('mangaki')
    user.email = '%d@mangaki.fr' % new_id
    user.date_joined = timezone.now()
    user.last_login = timezone.now()
    user.save()
    old_user = User.objects.get(id=old_id)
    old_user.rating_set.update(user=user)
    # old_user.profile.user = user
    old_user.delete()

In [29]:
User.objects.all()

[<User: 1960>, <User: 1971>, <User: 1992>, <User: 1956>, <User: 2060>, <User: 1998>, <User: 2070>, <User: 1999>, <User: 2024>, <User: 1988>, <User: 1976>, <User: 1986>, <User: 1935>, <User: 2013>, <User: 2087>, <User: 2002>, <User: 2054>, <User: 2088>, <User: 1972>, <User: 1946>, '...(remaining elements truncated)...']

In [30]:
import json
j = json.load(open('../fixtures/seed_data.json'))
Counter(entry['model'] for entry in j)

Counter({'auth.user': 200,
         'mangaki.anime': 20,
         'mangaki.artist': 54,
         'mangaki.category': 3,
         'mangaki.editor': 17,
         'mangaki.genre': 34,
         'mangaki.manga': 20,
         'mangaki.page': 4,
         'mangaki.rating': 3098,
         'mangaki.studio': 50,
         'mangaki.work': 40,
         'sessions.session': 1,
         'sites.site': 1})